In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

In [2]:
def query(url,parameter,interval,maxtrial):
    '''
    return json of requesting url with parameters, otherwise retry with an interval
    -----------------
    Parameters:
    -- url: string, API
    -- parameter: {'parameters' : 'value'}, parameters attached
    -- interval: int, time of sleep if request fail
    -- maxtrial: int, maximum number of attempt
    -----------------
    Returns:
    -- json
    '''
    if maxtrial == 0:
        print("request failed, stop trying!")
        return None
    response = requests.get(url=url,params=parameter)
    if response.status_code == 200:
        print('request successful!')
        return response.json()
    else:
        print('An error occurs! Retry after {} seconds'.format(interval))
        time.sleep(interval)
        return query(url,parameter,interval,maxtrial-1)
        
        

In [18]:
test = query(url='https://steamspy.com/api.php',parameter={"request":"all","page": "0"},interval=3,maxtrial=3)
print(len(test))

request successful!
1000


In [19]:
test

{'1134570': {'appid': 1134570,
  'name': 'F1 2021',
  'developer': 'Codemasters',
  'publisher': 'Electronic Arts',
  'score_rank': '',
  'positive': 29237,
  'negative': 4986,
  'userscore': 0,
  'owners': '1,000,000 .. 2,000,000',
  'average_forever': 5695,
  'average_2weeks': 549,
  'median_forever': 4005,
  'median_2weeks': 559,
  'price': '5999',
  'initialprice': '5999',
  'discount': '0',
  'ccu': 2107},
 '1088150': {'appid': 1088150,
  'name': 'Scribble It!',
  'developer': 'Detach Entertainment',
  'publisher': 'Detach Entertainment',
  'score_rank': '',
  'positive': 5374,
  'negative': 850,
  'userscore': 0,
  'owners': '1,000,000 .. 2,000,000',
  'average_forever': 154,
  'average_2weeks': 0,
  'median_forever': 64,
  'median_2weeks': 0,
  'price': '0',
  'initialprice': '0',
  'discount': '0',
  'ccu': 55},
 '1238860': {'appid': 1238860,
  'name': 'Battlefield 4',
  'developer': 'DICE',
  'publisher': 'Electronic Arts',
  'score_rank': '',
  'positive': 40075,
  'negative'

In [24]:
class appidExtractor:
    '''
    extract appid from given url
    Parameters:
    --- url: string, API
    --- name: string, data will be stored in name.csv
    --- interval: int, time of sleep if request fail
    '''
    def __init__(self, name, interval):
        '''
        -- csvpath: path of csv file
        -- tracepath: path of txt that stores pathidx
        -- trace: next page index to request. !!It is a string!!
        -- fieldnames: names of the columns to be written into .csv file
        '''
        csvfile = name+".csv"
        tracefile = name+"_trace.txt"
        self.interval = interval
        self.trace = 0
        self.fieldnames = []
        self.csvpath = os.path.join(os.getcwd(),csvfile)
        self.tracepath = os.path.join(os.getcwd(),tracefile)
        
    def writeheader(self):
        '''
        write header of csvfile based fieldnames
        '''
        with open(self.csvpath,mode='a',newline = '',encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames,extrasaction='ignore')
            writer.writeheader()
        return
        
    def initialize(self):
        '''
        initialize csv and page file if no previous request made
        otherwise, read page idx from tracefile
        if only one file exists, reset the progress
        '''
        try:
            csvfile = open(self.csvpath,'r')
            try:
                tracefile = open(self.tracepath,'r')
                self.trace = int(tracefile.read())
                tracefile.close()
                csvfile.close()
            except FileNotFoundError:
                csvfile.close()
                csvfile = open(self.csvpath,'w')
                csvfile.write("")
                csvfile.close()
                tracefile = open(self.tracepath,'w')
                tracefile.write("0")
                tracefile.close()
            except: 
                print("something else went wrong!")
                exit()
                
        except FileNotFoundError:
            csvfile = open(self.csvpath,'x')
            tracefile = open(self.tracepath,'w')
            tracefile.write("0")
            csvfile.close()
            tracefile.close()
        except:
            print("something else went wrong!")
            exit()
        
        self.writeheader()
        return
            
    def deleteAll(self):
        '''delete both csvfile and tracefile'''
        if os.path.exists(self.csvpath):
            os.remove(self.csvpath)
        if os.path.exists(self.tracepath):
            os.remove(self.tracepath)

        return
    
    def get_trace(self):
        '''
        read page idx from tracefile and set self.trace
        '''
        with open(self.tracepath,'r') as tracefile:
            self.trace = tracefile.read()
        return 
    
    def set_trace(self,val):
        '''
        set page idx in both self.trace and tracefile
        -- val: int
        '''
        self.trace = str(val)
        with open(self.tracepath,'w') as tracefile:
            tracefile.write(self.trace)
        return
    
    def set_fieldnames(self,fieldnames):
        '''
        set the names of the columns to be written into .csv file
        !!! must be set before run() !!!
        -- fieldnames: list of string
        '''
        self.fieldnames = fieldnames
        return
    
    
    def process_query(self):
        '''
        recursion to request data with all page idx
        '''
        self.get_trace()
        para = {"request":"all","page": self.trace}
        url = "https://steamspy.com/api.php"
        data = query(url,para,self.interval,3) # request data
        time.sleep(10)
        if data == None: # if request faied, exit
            exit()
        # write dictionary data into csv
        with open(self.csvpath, mode='a',newline = '',encoding='utf-8') as csvfile:
            fieldnames = ['appid', 'name']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames,extrasaction='ignore')
            for key in data:
                writer.writerow(data[key])
        print("successfully write {number} entries, page {trace}".format(number=len(data),trace=self.trace))
        # update trace locally
        self.set_trace(int(self.trace)+1)
        if len(data) < 1000: # check if it is the last page
            print("request finished")
            return
        else:
            return self.process_query()
        
        
    
    def run(self):
        '''
        request data from API
        '''
        if len(self.fieldnames) == 0:
            raise ValueError("fieldnames cannot be None") # if the fieldnames are not set, raise
        self.initialize()
        self.process_query()
        
        return
        
        

In [25]:
extractor = appidExtractor("appid",10)
extractor.set_fieldnames(['appid', 'name'])

In [26]:
extractor.deleteAll()

In [27]:
extractor.run()

request successful!
successfully write 1000 entries, page 0
request successful!
successfully write 1000 entries, page 1
request successful!
successfully write 1000 entries, page 2
request successful!
successfully write 1000 entries, page 3
request successful!
successfully write 1000 entries, page 4
request successful!
successfully write 1000 entries, page 5
request successful!
successfully write 1000 entries, page 6
request successful!
successfully write 1000 entries, page 7
request successful!
successfully write 1000 entries, page 8
request successful!
successfully write 1000 entries, page 9
request successful!
successfully write 1000 entries, page 10
request successful!
successfully write 1000 entries, page 11
request successful!
successfully write 1000 entries, page 12
request successful!
successfully write 1000 entries, page 13
request successful!
successfully write 1000 entries, page 14
request successful!
successfully write 1000 entries, page 15
request successful!
successfully w

In [28]:
appid_df = pd.read_csv("appid.csv")
appid_df.reset_index(inplace=True)
appid_df.sort_index(inplace=True)
appid_df

,index,appid,name
0,0,1887990,Deep Night Runner
1,1,680780,DEATH TRAIN - Warning: Unsafe VR Experience
2,2,1444740,Maniac Path
3,3,1619450,Heart of a Warrior
4,4,1401750,Mission Z
...,...,...,...
56991,56991,1506770,Crawler
56992,56992,1868280,Coins Collector Simulator
56993,56993,40370,Emergency 2012
56994,56994,1071880,New Yankee in King Arthur's Court 4


In [33]:
appid_df.loc[appid_df["appid"]==570]

,index,appid,name


In [ ]:
class detailExtractor(appidExtractor):
    def __init__(self, name, interval, parseID):
        super().__init__(name, interval)
        self.appidpath = os.path.join(os.getcwd(),"appid.csv")
        parselist = []
        self.parse = 
        
    def parseSteamspy(self):
        